### modifyParameters
- read in CLM parameter files and create a 'new' pft
- here copy arctic C3 grass for pft 11 and 12
- modify particular parameters for NWT simulations

Will Wieder Nov 2021

In [1]:
import xarray as xr
import pandas as pd
import os
import netCDF4
import numpy as np

### Parameter Files already modified to have 
- 10 day onset 
- 13h critDayLenth for offset


In [40]:
pathin = '/glade/work/wwieder/modified_inputs/NEON/'
basefile = pathin + 'ctsm51_params.c210803.NIWO_critDay13h_fastOnset.nc'
p = xr.open_dataset(basefile) 

In [41]:
mypft = 'c3_arctic_grass'
ix    = np.array([mypft in str(n) for n in p.pftname.values])  #index vector for mypft

param = 'medlynslope'
m = p[param].values
print(m[0:14])
print(m[ix])


[0.         2.3499999  2.3499999  2.3499999  4.11999989 4.11999989
 4.44999981 4.44999981 4.44999981 4.69999981 4.69999981 4.69999981
 2.22000003 5.25      ]
[2.22000003]


In [42]:
# Frist overwrite pft 13-79 with pft specific values for C3 Arctic grass
p2 = p.copy(deep=True)
for var_name, values in p.items():
    if p2[var_name].size == 79 :
        #print(var_name)
        p2[var_name].values[13:]=p2[var_name].values[12]

    # loop over phs parameters too
    if p2[var_name].size == 316 :
        print('phs '+var_name)
        for i in range(0,4):
            p2[var_name].values[i][13:]=p2[var_name].values[i][12]

    # loop over rootprof_beta parameters too
    if p2[var_name].size == 158 :
        print('rootprof '+var_name)
        for i in range(0,2):
            p2[var_name].values[i][13:]=p2[var_name].values[i][12]


phs ck
phs kmax
phs psi50
rootprof rootprof_beta


In [43]:
# Check to make sure it worked
var_name = 'slatop'
p2[var_name].isel(pft=slice(0,15))

<xarray.DataArray 'slatop' (pft: 15)>
array([0.     , 0.01   , 0.01   , 0.02018, 0.019  , 0.019  , 0.0308 , 0.0308 ,
       0.0308 , 0.01798, 0.03072, 0.028  , 0.021  , 0.021  , 0.021  ])
Coordinates:
    pftname  (pft) |S40 b'not_vegetated                           ' ... b'c4_...
Dimensions without coordinates: pft
Attributes:
    long_name:  Specific Leaf Area (SLA) at top of canopy, projected area basis
    units:      m^2/gC

---------------
#### parameter changes affecting all pfts
this won't place nicely with shrub simulations...

In [44]:
# Some additional parameter modifications for all simulations
p2['jmaxb0'].values = 0.02

#### Make specific changes for each pft
- 12 = Dry meadow
- 13 = Moist meadow
- 14 = Met meadow

In [45]:
p2['FUN_fracfixers'].values[12:15] = [0.05 ,0.05  ,0.05]
p2['slatop'].values[12:15]         = [0.0175,0.023,0.026] # previously used 0.018 from dry meadow, and no differences for wet meadow
p2['leafcn'].values[12:15]         = [21.5 ,19.5  ,17.5]  # prevously moist = wet meadow
p2['froot_leaf'].values[12:15]     = [2.5  ,1.5   ,1.5]   # prevously used 2.0 for dry meadow

In [46]:
# reduce kmax for dry meadow only
for i in range(0,4):
    p2['kmax'].values[i][12] = 2.3e-10
    print(p['kmax'].values[i][12])

2.4148408223121123e-09
2.4148408223121123e-09
2.4148408223121123e-09
2.4148408223121123e-09


In [ ]:
### now write out a .nc file

In [51]:
p2.to_netcdf(pathin + 'ctsm51_params.c210803.NIWO_critDay13h_fastOnset_c3ArcticMods.nc')